In [51]:
! pip -q install datasets
! pip install jsonargparse

# Timing test run in Colab!

Not on LUMI, probably does not work here

In [52]:
import datasets
import torch
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [53]:
dataset = datasets.load_dataset("TurkuNLP/genre-6", data_files="test.jsonl.gz")

In [54]:
model = SentenceTransformer("intfloat/multilingual-e5-large-instruct",trust_remote_code=True)

In [55]:
def using_datasets(dataset, model):
    dataset = dataset.map(lambda x:
                                        {"embeddings": model.encode(
                                                                    x["text"],  # No query here
                                                                    convert_to_tensor=False,
                                                                    normalize_embeddings=False
                                                                    )
                                        }
                                )
    return dataset

def using_pandas(df, model):
    input_texts = df["text"].tolist()
    embeddings = model.encode(input_texts, convert_to_tensor=False, normalize_embeddings=False)
    df["embeddings"] = [row.tolist() for row in embeddings]
    return df



def using_pandas_batch(df, model, batch_size=50):
    input_texts = df["text"].tolist()

    # Split the input texts into smaller batches
    input_text_batches = np.array_split(input_texts, len(input_texts) // batch_size + 1)

    embeddings = []

    # Process each batch
    for batch in input_text_batches:
        batch_embeddings = model.encode(batch, convert_to_tensor=False, normalize_embeddings=False)
        embeddings.extend(batch_embeddings)  # Accumulate the embeddings

    # Add the embeddings to the DataFrame
    df["embeddings"] = [row.tolist() for row in embeddings]

    return df

In [56]:

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'categories', '__index_level_0__', 'fiction', 'split1', 'split2', 'split3'],
        num_rows: 2554
    })
})


In [57]:
#hf_version = dataset["train"].select(range(10))
pd_version = dataset["train"].to_pandas().head(100)
#print(hf_version)
#print(pd_version)

In [58]:
#hf_version_e = using_datasets(hf_version, model)
pd_version_e = using_pandas(pd_version,model)
batch_version = using_pandas_batch(pd_version, model)

In [59]:
#print(hf_version_e["embeddings"][0])

In [60]:
print(pd_version_e["embeddings"][0])

[0.053849611431360245, -4.87484285258688e-05, -0.018496278673410416, -0.007329392246901989, 0.02551181986927986, -0.06001888960599899, -0.0326429083943367, 0.06594373285770416, 0.04784242808818817, -0.0028862664476037025, 0.02896403707563877, 0.02006112039089203, -0.039150696247816086, -0.037932466715574265, 0.00122923799790442, -0.01566321775317192, -0.051005154848098755, 0.03213897719979286, -0.02392733283340931, -0.021218454465270042, 0.0628790333867073, -0.018576201051473618, -0.05222763493657112, -0.03677593171596527, -0.024877114221453667, -0.02636522240936756, -0.022740503773093224, -0.040163200348615646, -0.0447612926363945, -0.028134465217590332, -0.008967792615294456, 0.03841851279139519, -0.04687885195016861, -0.027545766904950142, -0.03757671266794205, 0.04129067435860634, 0.015585134737193584, 0.04902522265911102, -0.05774107947945595, 0.038720596581697464, -0.007221539970487356, 0.013261050917208195, 0.01512469258159399, -0.05393015965819359, -0.01801970601081848, 0.01376

In [61]:
print(batch_version["embeddings"][0])

[0.053849611431360245, -4.87484285258688e-05, -0.018496278673410416, -0.007329392246901989, 0.02551181986927986, -0.06001888960599899, -0.0326429083943367, 0.06594373285770416, 0.04784242808818817, -0.0028862664476037025, 0.02896403707563877, 0.02006112039089203, -0.039150696247816086, -0.037932466715574265, 0.00122923799790442, -0.01566321775317192, -0.051005154848098755, 0.03213897719979286, -0.02392733283340931, -0.021218454465270042, 0.0628790333867073, -0.018576201051473618, -0.05222763493657112, -0.03677593171596527, -0.024877114221453667, -0.02636522240936756, -0.022740503773093224, -0.040163200348615646, -0.0447612926363945, -0.028134465217590332, -0.008967792615294456, 0.03841851279139519, -0.04687885195016861, -0.027545766904950142, -0.03757671266794205, 0.04129067435860634, 0.015585134737193584, 0.04902522265911102, -0.05774107947945595, 0.038720596581697464, -0.007221539970487356, 0.013261050917208195, 0.01512469258159399, -0.05393015965819359, -0.01801970601081848, 0.01376

In [62]:
#for row1, (i, row2) in zip(hf_version_e, pd_version_e.iterrows()):
    #print(row1["embeddings"])
    #print(row2["embeddings"])
#    print(all([abs(j-k)<1e-6 for j,k in zip(row1["embeddings"], row2["embeddings"])]))

for (l,row1), (i, row2) in zip(batch_version.iterrows(), pd_version_e.iterrows()):
    #print(row1["embeddings"])
    #print(row2["embeddings"])
    print(all([abs(j-k)<1e-9 for j,k in zip(row1["embeddings"], row2["embeddings"])]))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [63]:
import time
a = time.time()
_ = using_pandas(pd_version, model)
b = time.time()
print(b-a)


45.450855016708374


In [64]:
a = time.time()
_ = using_pandas_batch(pd_version,model)
b = time.time()
print(b-a)

49.09247922897339
